In [5]:
import os
import numpy as np
import tensorflow as tf

In [2]:
n_samples = 1_000
n_features = 7
x = np.random.randn(n_samples, n_features)
names = np.random.randint(0, 100, size=n_samples).astype(str)
x, names

(array([[-1.02330989,  1.10112117,  0.26816211, ..., -1.93717991,
         -1.23120198,  1.75488616],
        [-0.53454485,  1.09275624, -0.69104461, ..., -0.69627258,
          0.19419093, -1.66526972],
        [-0.80634423, -0.21945347, -0.65996497, ...,  0.0094244 ,
          0.41984597,  1.61488729],
        ...,
        [-0.84350663, -0.42786854,  0.73615638, ...,  0.50393727,
         -0.67110885,  1.20727151],
        [ 0.87365795,  0.63888648,  1.63800232, ..., -1.47422034,
          0.63167284,  1.18165092],
        [ 1.24367518, -0.16088102, -0.54212906, ...,  0.67514079,
         -0.21209837,  0.97833255]]),
 array(['49', '45', '18', '68', '36', '21', '98', '60', '31', '79', '93',
        '65', '32', '50', '96', '65', '65', '25', '65', '61', '62', '12',
        '38', '74', '66', '11', '93', '18', '37', '94', '44', '24', '30',
        '54', '57', '14', '63', '66', '13', '35', '14', '43', '87', '13',
        '80', '61', '7', '5', '63', '39', '99', '81', '83', '20', '97',
     

In [13]:
example_path = 'example.tfrecords'
# Write the records to a file.
with tf.io.TFRecordWriter(example_path) as file_writer:
    for i in range(n_samples):
        record_bytes = tf.train.Example(
            features=tf.train.Features(feature={
                "f": tf.train.Feature(float_list=tf.train.FloatList(value=x[i])),
                "name": tf.train.Feature(bytes_list=tf.train.BytesList(value=[names[i].encode('UTF-8')])),
            })).SerializeToString()
        file_writer.write(record_bytes)

In [18]:
def tfrecord_decoder(record_bytes):
    return tf.io.parse_single_example(
        record_bytes,
        
        # schema
        {'f': tf.io.FixedLenFeature([n_features], dtype=tf.float32),
         'name': tf.io.FixedLenFeature([], dtype=tf.string)}
    )

In [19]:
for batch in tf.data.TFRecordDataset([example_path]).map(tfrecord_decoder):
    print(batch)
    break

{'f': <tf.Tensor: shape=(7,), dtype=float32, numpy=
array([-1.02331   ,  1.1011212 ,  0.2681621 ,  0.37342593, -1.9371799 ,
       -1.231202  ,  1.7548862 ], dtype=float32)>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'49'>}


 real data

In [2]:
def tfrecord_decoder(record_bytes):
    parsed_example = tf.io.parse_single_example(
        record_bytes,
        
        # schema
        {'layout_id': tf.io.FixedLenFeature([], dtype=tf.string),
         'config_index': tf.io.FixedLenFeature([], dtype=tf.int64),
         'node_descriptor': tf.io.FixedLenFeature([], dtype=tf.string),
         'valid_nodes': tf.io.FixedLenFeature([], dtype=tf.int64),
         'graph_descriptor': tf.io.FixedLenFeature([121], dtype=tf.float32),
         'target': tf.io.FixedLenFeature([], dtype=tf.float32)
        }
    )
    parsed_example['node_descriptor'] = tf.io.parse_tensor(
        parsed_example['node_descriptor'], tf.float32
    )
    return parsed_example


In [3]:
for i, batch in enumerate(tf.data.TFRecordDataset(
    ['layout_tfrecords/layout:xla:default:mask_rcnn_resnet50.4x4.bf16.performance:train.tfrecords'],
    compression_type="GZIP").map(tfrecord_decoder)):
    print(batch['config_index'].numpy())
    if i == 20:
        print(batch)
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
{'config_index': <tf.Tensor: shape=(), dtype=int64, numpy=20>, 'graph_descriptor': <tf.Tensor: shape=(121,), dtype=float32, numpy=
array([5.7150115e-04, 1.4204979e-01, 0.0000000e+00, 0.0000000e+00,
       3.3274066e-02, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 2.4130049e-03,
       8.1407160e-02, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       9.5250190e-04, 0.0000000e+00, 2.5400051e-04, 2.1082042e-02,
       0.0000000e+00, 3.3655067e-03, 0.0000000e+00, 2.5209550e-02,
       4.3688089e-02, 1.6510032e-02, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 1.9685039e-03, 3.2385064e-03,
       0.0000000e+00, 8.2550163e-04, 6.3500128e-04, 0.0000000e+00,
       1.0795022e-03, 0.0000000e+00, 0.0000000e+00, 5.0800102e-04,
       2.2860046e-03, 2.0955042e-03, 0.0000000e+00, 0.0000000e+00,
       5.0482601e-02, 0.0000000e+00, 0.0000000e+00, 2.9210059e-03,
       0.000

2023-09-30 02:11:15.232035: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-30 02:11:15.322063: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-30 02:11:15.322241: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [12]:
tfrecords_file_list = os.listdir('layout_tfrecords/')
nlp_train_files = []
for filename in tfrecords_file_list:
    f = filename[:-(len('.tfrecords'))].split(':')
    if f[-1] == 'train' and f[1] == 'xla':
        nlp_train_files.append('layout_tfrecords/'+filename)
nlp_train_files

['layout_tfrecords/layout:xla:random:resnet50.4x4.bf16.performance:train.tfrecords',
 'layout_tfrecords/layout:xla:default:mask_rcnn_batch_16_bf16_img1024:train.tfrecords',
 'layout_tfrecords/layout:xla:random:inference_mlperf_ssd_1200_batch_1:train.tfrecords',
 'layout_tfrecords/layout:xla:random:resnet50_3d.2x2.bf16:train.tfrecords',
 'layout_tfrecords/layout:xla:random:bert_squad.2x2.fp32:train.tfrecords',
 'layout_tfrecords/layout:xla:random:mlperf_resnet:train.tfrecords',
 'layout_tfrecords/layout:xla:default:alexnet_train_batch_32:train.tfrecords',
 'layout_tfrecords/layout:xla:default:xception_imagenet:train.tfrecords',
 'layout_tfrecords/layout:xla:random:unet3d:train.tfrecords',
 'layout_tfrecords/layout:xla:default:bert_classifier.2x2.fp32:train.tfrecords',
 'layout_tfrecords/layout:xla:random:shapemask.4x4.fp32:train.tfrecords',
 'layout_tfrecords/layout:xla:default:resnet50.4x4.bf16:train.tfrecords',
 'layout_tfrecords/layout:xla:default:mask_rcnn_resnet50.4x4.bf16.performa

In [15]:
dataset = tf.data.Dataset.from_tensor_slices(nlp_train_files)
dataset = dataset.interleave(lambda x: tf.data.TFRecordDataset(x, compression_type='GZIP'),
    cycle_length=4, num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=False)
dataset = dataset.map(tfrecord_decoder)

In [18]:
for b in dataset:
    print(b)
    break

{'config_index': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'graph_descriptor': <tf.Tensor: shape=(121,), dtype=float32, numpy=
array([1.0493179e-03, 9.8560937e-02, 0.0000000e+00, 1.4990257e-04,
       1.2366962e-02, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 4.4221254e-03,
       7.3752061e-02, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       3.7475640e-04, 0.0000000e+00, 7.4951284e-05, 3.2978565e-02,
       0.0000000e+00, 2.0986358e-03, 0.0000000e+00, 2.1286163e-02,
       4.4970769e-02, 1.6639184e-02, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 3.7475640e-04, 3.1479539e-03,
       0.0000000e+00, 1.0493179e-03, 7.4951281e-04, 0.0000000e+00,
       1.3491231e-03, 0.0000000e+00, 0.0000000e+00, 5.9961027e-04,
       3.2229051e-03, 2.9980512e-03, 0.0000000e+00, 0.0000000e+00,
       2.8991157e-01, 0.0000000e+00, 7.4951284e-05, 7.9448363e-03,
       0.0000000e+00, 3.7475640e-04, 4.4970770e-04, 4.3471744e-03,